In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  # 输入通道1，输出通道6，卷积核大小5
        self.pool = nn.MaxPool2d(2, 2)  # 最大池化，核大小2，步长2
        self.conv2 = nn.Conv2d(6, 16, 5)  # 输入通道6，输出通道16，卷积核大小5
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 全连接层，输入维度16*4*4，输出维度120
        self.fc2 = nn.Linear(120, 84)  # 全连接层，输入维度120，输出维度84
        self.fc3 = nn.Linear(84, 10)  # 全连接层，输入维度84，输出维度10

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 第一层卷积 -> ReLU -> 池化
        x = self.pool(F.relu(self.conv2(x)))  # 第二层卷积 -> ReLU -> 池化
        x = x.view(-1, 16 * 4 * 4)  # 展平操作，准备全连接
        x = F.relu(self.fc1(x))  # 第一层全连接 -> ReLU
        x = F.relu(self.fc2(x))  # 第二层全连接 -> ReLU
        x = self.fc3(x)  # 第三层全连接
        return x

# 加载数据集
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# 初始化网络和优化器
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# 训练网络
for epoch in range(2):  # 多次遍历数据集
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  # 每2000个批次打印一次
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.133
[1,  4000] loss: 0.207
[1,  6000] loss: 0.153
[1,  8000] loss: 0.104
[1, 10000] loss: 0.096
[1, 12000] loss: 0.089
[1, 14000] loss: 0.080
[2,  2000] loss: 0.062
[2,  4000] loss: 0.061
[2,  6000] loss: 0.058
[2,  8000] loss: 0.064
[2, 10000] loss: 0.053
[2, 12000] loss: 0.059
[2, 14000] loss: 0.052
Finished Training
